In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL to scrape
url = "https://samlingar.shm.se/sok?type=object&query=Vikingatid&listType=archaeological&rows=1000&offset=0"
driver.get(url)

# Function to scroll to the bottom of the page
def scroll_to_bottom(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  # Wait for new content to load

# Scroll to the bottom of the page until no new content is loaded
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    scroll_to_bottom(driver)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Wait for the table to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "table"))
)

# Scrape the table
try:
    table = pd.read_html(driver.page_source)[0]
    print(table.head())  # Display the first few rows of the table
except Exception as e:
    print("Error scraping table:", e)

driver.quit()


  Bild Föremålsbenämning      Föremålsnr.          Förvärvsnr.  \
0  NaN  yxa med skafthål  TESTPOST_SIS_01  TESTPOST_SIS_ACC_01   
1  NaN              Mynt         2720_SKO                    -   
2  NaN             Svärd        13389_SKO                    -   
3  NaN          Skäggyxa        13391_SKO                    -   
4  NaN  Svärd Petersen H        13394_SKO                    -   

                Andra nummer  Material  \
0  Nr 1867: 17:17:17:896:783  Päronträ   
1                          -    Silver   
2                          -      Järn   
3                          -      Järn   
4                          -      Järn   

                                               Plats  \
0  Ohio, San Juan County, Reapture Creek, Fagersk...   
1                                                  -   
2                                                  -   
3                                                  -   
4                                                  -   

             

In [3]:
table.to_csv('Viking_artifacts_big.csv')

In [5]:
table

,Bild,Föremålsbenämning,Föremålsnr.,Förvärvsnr.,Andra nummer,Material,Plats,Fornlämning,Socken,Landskap,Land,Kontexttyp,Kontextnr.,Artbedömning,Benslagsbedömning,Museum
0,NaN,yxa med skafthål,TESTPOST_SIS_01,TESTPOST_SIS_ACC_01,Nr 1867: 17:17:17:896:783,Päronträ,"Ohio, San Juan County, Reapture Creek, Fagersk...",TESTPOST_SIS_GEO_FORNL_02,-,-,-,Boplats,09876_TEST,Accipiter nisus,Membrum superius,NaN
1,NaN,Mynt,2720_SKO,-,-,Silver,-,-,-,-,-,-,-,-,-,NaN
2,NaN,Svärd,13389_SKO,-,-,Järn,-,-,-,-,-,-,-,-,-,NaN
3,NaN,Skäggyxa,13391_SKO,-,-,Järn,-,-,-,-,-,-,-,-,-,NaN
4,NaN,Svärd Petersen H,13394_SKO,-,-,Järn,-,-,-,-,-,-,-,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,Mynt,3044996,104626,FID: 1253086,Silver,S:t Olofs konvent,L2010:31,Skänninge socken,Östergötland,Sverige,Grav,70017,-,-,NaN
496,NaN,Mynt,3037672,27130,Undernummer: 1,Silver,Källa ödekyrka,-,Källa socken,Öland,Sverige,-,-,-,-,NaN
497,NaN,Mynt,3199239,103506,Undernummer: c,Silver,Stora Sojdeby,L1977:5513,Fole socken,Gotland,Sverige,-,-,-,-,NaN
498,NaN,Mynt,3038597,100508,Undernummer: 13,Silver,Källa ödekyrka,-,Källa socken,Öland,Sverige,-,-,-,-,NaN


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

# Initialize the web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL to scrape
url = "https://samlingar.shm.se/sok?type=object&query=Vikingatid&listType=archaeological&rows=1000&offset=0"
driver.get(url)

# Wait for the initial content to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "table"))
)

# Scrape the table
try:
    table = pd.read_html(driver.page_source)[0]
except Exception as e:
    print("Error scraping table:", e)
    driver.quit()
    raise

# Drop the 'Bild' column
if 'Bild' in table.columns:
    table.drop(columns=['Bild'], inplace=True)

# Extract museum names from the 'title' attribute of each <i> element
museum_elements = driver.find_elements(By.CSS_SELECTOR, "td i.museum-icon")
museum_names = [elem.get_attribute('title') for elem in museum_elements]

# Replace the 'Museum' column with extracted text
if 'Museum' in table.columns and len(museum_names) == len(table):
    table['Museum'] = museum_names
else:
    print("Mismatch in number of rows while extracting museum names")

driver.quit()


In [15]:

# Replace '-' with NaNs in the entire DataFrame
table.replace('-', np.NaN, inplace=True)

table.head()  # Display the first few rows of the table


,Föremålsbenämning,Föremålsnr.,Förvärvsnr.,Andra nummer,Material,Plats,Fornlämning,Socken,Landskap,Land,Kontexttyp,Kontextnr.,Artbedömning,Benslagsbedömning,Museum
0,yxa med skafthål,TESTPOST_SIS_01,TESTPOST_SIS_ACC_01,Nr 1867: 17:17:17:896:783,Päronträ,"Ohio, San Juan County, Reapture Creek, Fagersk...",TESTPOST_SIS_GEO_FORNL_02,NaN,NaN,NaN,Boplats,09876_TEST,Accipiter nisus,Membrum superius,Historiska museet
1,Mynt,2720_SKO,NaN,NaN,Silver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skoklosters slott
2,Svärd,13389_SKO,NaN,NaN,Järn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skoklosters slott
3,Skäggyxa,13391_SKO,NaN,NaN,Järn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skoklosters slott
4,Svärd Petersen H,13394_SKO,NaN,NaN,Järn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Skoklosters slott


In [ ]:

table.to_csv('Viking_artifacts_big.csv')
